# Repurposhing Antminer S9 Control Board as FPGA Developmetn Board
- SoC ZYNQ 7010 (Dual Core ARM Cortex-A9 + FPGA Artix-7 28.000 Logic Cells) <br><br>
<img src="resource/Antminer S9.jpg" width="900px">


### 1. Prerequisites
- Antminer S9 Control Board
- SD Card flashed with PYNQ 2.5
    - Can be downloaded from : [https://github.com/cnchens/s9-v1.2-pynq2.5/tree/master](https://github.com/cnchens/s9-v1.2-pynq2.5/tree/master)
    - Flash using Balena Etcher
- Install `AntminerGPIO` Overlay from [https://github.com/guannan-he/Antminer_s9_pynq/tree/main](https://github.com/guannan-he/Antminer_s9_pynq/tree/main)

### 2. Load Overlay

In [ ]:
from pynq import Overlay
from pynq.lib import AxiGPIO
import time

In [2]:
gpio = Overlay('antminerGPIO.bit')

- Hardware design for 'antminerGPIO.bit'<br><br>
<img src="https://raw.githubusercontent.com/guannan-he/Antminer_s9_pynq/main/antminerGPIO/pics/block_design_base.png" width="700px">

### 3. Play with On-board LED
- Using Xilinx AXI GPIO 
    - ip block `"plLED"`

In [3]:
led_instance = gpio.ip_dict["plLED"]
leds = AxiGPIO(led_instance).channel1

In [7]:
leds[0:4].write(0b1110)  # first led enlightened(near eth port)

In [9]:
leds[0:4].write(0b0111)  # last led enlightened

In [ ]:
# running 4 led

i = 0
while i < 100 :
    j = i % 8
    if j < 4:
        value = 0b1111 >> j  # This shifts the bits to the right
    else:
        value = 0b1111 >> (7 - j) # This reverses the pattern after 0000
    leds[0:4].write(value) 
    time.sleep(0.125)
    i += 1

In [12]:
leds[0:4].off()  # turn on all leds

In [13]:
leds[0:4].on()  # shut down all leds

<br><br><br><br>
### 4. Playing with GPIO in Port 0 - 9
- Using Xilinx AXI GPIO 
    - ip block `"plMiner0"` - `"plMiner9"`
<br><br>
<img src="resource/Antminer S9 Pin Mapping.png" width="1000px">

In [15]:
miner_instance = gpio.ip_dict["plMiner0"]
miner0 = AxiGPIO(miner_instance).channel1

- `miner0[0:4].write()` will `write` digital state into `pin 3`, `pin 2`, `pin 1`, `pin 0` on `"plMiner0"`
- `miner0[0].write()` will `write` digital state into `pin 3` on `"plMiner0"`
- `miner0[1].write()` will `write` digital state into `pin 2` on `"plMiner0"`
- `miner0[2].write()` will `write` digital state into `pin 1` on `"plMiner0"`
- `miner0[3].write()` will `write` digital state into `pin 0` on `"plMiner0"`

In [23]:
miner0[0:4].write(0b1111) # turn all miner0 output to HIGH

In [28]:
miner0[0:4].write(0b0000) # turn all miner0 output to LOW

<br><br><br><br>
### 5. Playing with 7 Segment
- Wiring diagram <br><br>
<img src="resource/7Segment Wiring.png" width="700px">

In [24]:
gpio_7segment_a_instant = gpio.ip_dict["plMiner1"]
gpio_7segment_a = AxiGPIO(gpio_7segment_a_instant).channel1

gpio_7segment_b_instant = gpio.ip_dict["plMiner2"]
gpio_7segment_b = AxiGPIO(gpio_7segment_b_instant).channel1

In [25]:
# turn all 7 segment element to ON
gpio_7segment_a[0:4].write(0b1111) # d c b a
gpio_7segment_b[0:4].write(0b1111) # dp h g f

In [ ]:
# 7 segmnet counting up and down

seven_segment_config = {
    0: (0b1111, 0b0011),  # Display '0'
    1: (0b0110, 0b0000),  # Display '1'
    2: (0b1011, 0b0101),  # Display '2'
    3: (0b1111, 0b0100),  # Display '3'
    4: (0b0110, 0b0110),  # Display '4'
    5: (0b1101, 0b0110),  # Display '5'
    6: (0b1101, 0b0111),  # Display '6'
    7: (0b0111, 0b0000),  # Display '7'
    8: (0b1111, 0b0111),  # Display '8'
    9: (0b1111, 0b0110),  # Display '9'
}

iterator = None
for j in range(10000) :
    iterator = range(9, -1, -1) if j % 2 else range(1, 9)
    
    for i in iterator :
        value = seven_segment_config[i]
        gpio_7segment_a[0:4].write(value[0])
        gpio_7segment_b[0:4].write(value[1])

        time.sleep(0.25)

In [27]:
# turn all 7 segment element to OFF
gpio_7segment_a[0:4].write(0b0000) 
gpio_7segment_b[0:4].write(0b0000) 

<br><br><br><br>
### 6. Playing with TM1638 7 Segment Display Keypad & LED Module
- Wiring diagram <br><br>
<img src="resource/TM1638.png" width="900px">

In [29]:
TM1638_CMD1 = 0x40   # 0x40 data command
TM1638_CMD2 = 0xC0   # 0xC0 address command
TM1638_CMD3 = 0x80   # 0x80 display control command
TM1638_DSP_ON = 0x08 # 0x08 display on
TM1638_READ = 0x02   # 0x02 read key scan data
TM1638_FIXED = 0x04  # 0x04 fixed address mode

# 0-9, a-z, blank, dash, star
_SEGMENTS = bytearray(b'\x3F\x06\x5B\x4F\x66\x6D\x7D\x07\x7F\x6F\x77\x7C\x39\x5E\x79\x71\x3D\x76\x06\x1E\x76\x38\x55\x54\x3F\x73\x67\x50\x6D\x78\x3E\x1C\x2A\x76\x6E\x5B\x00\x40\x63')

In [30]:
class TM1638(object):
    """
    Library for the TM1638 LED display driver.
    Original author : https://github.com/mcauser/micropython-tm1638
    Modified by : https://github.com/Muhammad-Yunus
    """
    def __init__(self, stb_pin=0, clk_pin=3, dio_pin=2, pl_port="plMiner3", pl_overlay='antminerGPIO.bit', brightness=7):

        self._stb_pin = stb_pin
        self._clk_pin = clk_pin
        self._dio_pin = dio_pin
        self._pl_port = pl_port
        self._pl_overlay = pl_overlay
        
        self._gpio = Overlay(self._pl_overlay)
        self._tm1638 = AxiGPIO(self._gpio.ip_dict[self._pl_port]).channel1

        if not 0 <= brightness <= 7:
            raise ValueError("Brightness out of range")
        self._brightness = brightness

        self._on = TM1638_DSP_ON

        self.clk(1)
        self.dio(0)
        self.stb(1)
        
        self.clear()
        self._write_dsp_ctrl()
        
    def stb(self, value):
        self._tm1638[self._stb_pin:self._stb_pin+1].write(value)
        
    def clk(self, value):
        self._tm1638[self._clk_pin:self._clk_pin+1].write(value)
      
    def dio(self, value):
        self._tm1638[self._dio_pin:self._dio_pin+1].write(value)
    
    def _write_data_cmd(self):
        # data command: automatic address increment, normal mode
        self._command(TM1638_CMD1)

    def _set_address(self, addr=0):
        # address command: move to address
        self._byte(TM1638_CMD2 | addr)

    def _write_dsp_ctrl(self):
        # display command: display on, set brightness
        self._command(TM1638_CMD3 | self._on | self._brightness)

    def _command(self, cmd):
        self.stb(0)
        self._byte(cmd)
        self.stb(1)

    def _byte(self, b):
        for i in range(8):
            self.clk(0)
            self.dio((b >> i) & 1)
            self.clk(1)

    def _scan_keys(self):
        """Reads one of the four bytes representing which keys are pressed."""
        pressed = 0
        self.dio.init(Pin.IN, Pin.PULL_UP)
        for i in range(8):
            self.clk(0)
            if self.dio.value():
                pressed |= 1 << i
            self.clk(1)
        self.dio.init(Pin.OUT)
        return pressed

    def power(self, val=None):
        """Power up, power down or check status"""
        if val is None:
            return self._on == TM1638_DSP_ON
        self._on = TM1638_DSP_ON if val else 0
        self._write_dsp_ctrl()

    def brightness(self, val=None):
        """Set the display brightness 0-7."""
        # brightness 0 = 1/16th pulse width
        # brightness 7 = 14/16th pulse width
        if val is None:
            return self._brightness
        if not 0 <= val <= 7:
            raise ValueError("Brightness out of range")
        self._brightness = val
        self._write_dsp_ctrl()

    def clear(self):
        """Write zeros to each address"""
        self._write_data_cmd()
        self.stb(0)
        self._set_address(0)
        for i in range(16):
            self._byte(0x00)
        self.stb(1)

    def write(self, data, pos=0):
        """Write to all 16 addresses from a given position.
        Order is left to right, 1st segment, 1st LED, 2nd segment, 2nd LED etc."""
        if not 0 <= pos <= 15:
            raise ValueError("Position out of range")
        self._write_data_cmd()
        self.stb(0)
        self._set_address(pos)
        for b in data:
            self._byte(b)
        self.stb(1)

    def led(self, pos, val):
        """Set the value of a single LED"""
        self.write([val], (pos << 1) + 1)

    def leds(self, val):
        """Set all LEDs at once. LSB is left most LED.
        Only writes to the LED positions (every 2nd starting from 1)"""
        self._write_data_cmd()
        pos = 1
        for i in range(8):
            self.stb(0)
            self._set_address(pos)
            self._byte((val >> i) & 1)
            pos += 2
            self.stb(1)

    def segments(self, segments, pos=0):
        """Set one or more segments at a relative position.
        Only writes to the segment positions (every 2nd starting from 0)"""
        if not 0 <= pos <= 7:
            raise ValueError("Position out of range")
        self._write_data_cmd()
        for seg in segments:
            self.stb(0)
            self._set_address(pos << 1)
            self._byte(seg)
            pos += 1
            self.stb(1)

    def keys(self):
        """Return a byte representing which keys are pressed. LSB is SW1"""
        keys = 0
        self.stb(0)
        self._byte(TM1638_CMD1 | TM1638_READ)
        for i in range(4):
            keys |= self._scan_keys() << i
        self.stb(1)
        return keys

    def qyf_keys(self):
        """Return a 16-bit value representing which keys are pressed. LSB is SW1"""
        keys = 0
        self.stb(0)
        self._byte(TM1638_CMD1 | TM1638_READ)
        for i in range(4):
            i_keys = self._scan_keys()
            for k in range(2):
                for j in range(2):
                    x = (0x04 >> k) << j*4
                    if i_keys & x == x:
                        keys |= (1 << (j + k*8 + 2*i))
        self.stb(1)
        return keys

    def encode_digit(self, digit):
        """Convert a character 0-9, a-f to a segment."""
        return _SEGMENTS[digit & 0x0f]

    def encode_string(self, string):
        """Convert an up to 8 character length string containing 0-9, a-z,
        space, dash, star to an array of segments, matching the length of the
        source string excluding dots, which are merged with previous char."""
        segments = bytearray(len(string.replace('.','')))
        j = 0
        for i in range(len(string)):
            if string[i] == '.' and j > 0:
                segments[j-1] |= (1 << 7)
                continue
            segments[j] = self.encode_char(string[i])
            j += 1
        return segments

    def encode_char(self, char):
        """Convert a character 0-9, a-z, space, dash or star to a segment."""
        o = ord(char)
        if o == 32:
            return _SEGMENTS[36] # space
        if o == 42:
            return _SEGMENTS[38] # star/degrees
        if o == 45:
            return _SEGMENTS[37] # dash
        if o >= 65 and o <= 90:
            return _SEGMENTS[o-55] # uppercase A-Z
        if o >= 97 and o <= 122:
            return _SEGMENTS[o-87] # lowercase a-z
        if o >= 48 and o <= 57:
            return _SEGMENTS[o-48] # 0-9
        raise ValueError("Character out of range: {:d} '{:s}'".format(o, chr(o)))

    def hex(self, val):
        """Display a hex value 0x00000000 through 0xffffffff, right aligned, leading zeros."""
        string = '{:08x}'.format(val & 0xffffffff)
        self.segments(self.encode_string(string))

    def number(self, num):
        """Display a numeric value -9999999 through 99999999, right aligned."""
        # limit to range -9999999 to 99999999
        num = max(-9999999, min(num, 99999999))
        string = '{0: >8d}'.format(num)
        self.segments(self.encode_string(string))

    #def float(self, num):
    #    # needs more work
    #    string = '{0:>9f}'.format(num)
    #    self.segments(self.encode_string(string[0:9]))

    def temperature(self, num, pos=0):
        """Displays 2 digit temperature followed by degrees C"""
        if num < -9:
            self.show('lo', pos) # low
        elif num > 99:
            self.show('hi', pos) # high
        else:
            string = '{0: >2d}'.format(num)
            self.segments(self.encode_string(string), pos)
        self.show('*C', pos + 2) # degrees C

    def humidity(self, num, pos=4):
        """Displays 2 digit humidity followed by RH"""
        if num < -9:
            self.show('lo', pos) # low
        elif num > 99:
            self.show('hi', pos) # high
        else:
            string = '{0: >2d}'.format(num)
            self.segments(self.encode_string(string), pos)
        self.show('rh', pos + 2) # relative humidity

    def show(self, string, pos=0):
        """Displays a string"""
        segments = self.encode_string(string)
        self.segments(segments[:8], pos)

    def scroll(self, string, delay=250):
        """Display a string, scrolling from the right to left, speed adjustable.
        String starts off-screen right and scrolls until off-screen left."""
        segments = string if isinstance(string, list) else self.encode_string(string)
        data = [0] * 16
        data[8:0] = list(segments)
        for i in range(len(segments) + 9):
            self.segments(data[0+i:8+i])
            time.sleep(delay/1000)

In [31]:
# instantiate the TM1638 class
tm = TM1638(brightness=7)

In [ ]:
while True :
    # turn them on, one by one
    for i in range(8):
        tm.led(i, 1)
        time.sleep(0.2)

    # turn them off, one by one
    for i in range(8):
        tm.led(i, 0)
        time.sleep(0.2)

In [33]:
# display "01234567" using bytes from the tm1638._SEGMENTS font
tm.segments([0x3F,0x06,0x5B,0x4F,0x66,0x6D,0x7D,0x07])

In [34]:
# display "abcdefgh"
tm.segments(tm.encode_string('abcdefgh'))

In [35]:
# there is a .show() method for simplifying this
tm.show('abcdefgh')

In [36]:
# show supports a position offset, so you can insert characters anywhere in the 8x segments
tm.show('abcdefg', 1)
# tm.show('abcd')
# tm.show('efgh', 4)

In [37]:
# you can write blank segments by using spaces
tm.show('   123.45')

In [38]:
# a dot trailing a supported character will get merged with the character as the decimal place
# tm.show('a.b.cdefgh')
# tm.show('a.b.c.d.e.f.g.h.')
tm.show('0.0000000')
# tm.show('0.0.0.0.0000')

In [ ]:
# the scroll method accepts any length string and displays it by scrolling in from the right until it is completely offscreen on the left
while True :
    tm.scroll('cool', 150)
    # tm.scroll('4 FPS', 250)
    # tm.scroll('faster', 125)
    # tm.scroll('slower', 500)